## Working with SQL using Python

The Amazon reviews dataset provided a great opportunity for me to learn how to perform ETL processes using Python.



**We will be working with the 'pyodbc' package**

In [1]:
import os
import numpy as np
import pandas as pd
import pyodbc
import gzip
import json


**Setting the connection**


In [3]:
sql_conn = pyodbc.connect('DRIVER={ODBC Driver 13 for SQL Server};SERVER=.;DATABASE=AMZN_REV;Trusted_Connection=yes')


**Setting the path**


In [4]:
path = "C:\\Users\\byyre\\Desktop\\Projects\\Time Series Project\\Raw files"


**Defining the functions that will use to access the json files**


In [6]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')


**Query the data**


In [9]:
query = "SELECT * FROM [AMZN_REV].[dbo].[baby_reviews]"
df = pd.read_sql(query, sql_conn)
df.head()

,PK,reviewerID,asin,helpful,overall,reviewTime
0,1,A1HK2FQW6KXQB2,097293751X,"[0, 0]",5,"07 16, 2013"
1,2,A19K65VY14D13R,097293751X,"[0, 0]",5,"06 29, 2013"
2,3,A2LL1TGG90977E,097293751X,"[0, 0]",5,"03 19, 2014"
3,4,A5G19RYX8599E,097293751X,"[0, 0]",5,"08 17, 2013"
4,5,A2496A4EWMLQ7,097293751X,"[0, 0]",4,"04 1, 2014"



**Creating list of files and a dictionary of Dataframes for the loop to iterate on**


In [8]:
files_list = []
for i in os.listdir(path):
    files_list.append(i)

d = {}
for name in files_list:
    d[name[:14]] = getDF(path + '\\' + name)


**First step, creating the tables**


In [10]:
cursor = sql_conn.cursor()
for value, key in d.items():
    string = "CREATE TABLE {} (PK INT PRIMARY KEY IDENTITY, reviewerID NVARCHAR(100), asin NVARCHAR(100), helpful NVARCHAR(20), overall INT, reviewTime NVARCHAR(50))".format(value)
    cursor.execute(string)
    cursor.commit()


**Second step, creating nonclustered indexes**


In [13]:
cursor = sql_conn.cursor()
for value, key in d.items():
    string = "CREATE INDEX DATE ON [AMZN_REV].[dbo].[{}](reviewTime)".format(value)
    cursor.execute(string)
    cursor.commit()


**last step, inserts**


In [11]:
for value, key in d.items():
    key.to_csv(value + '.csv')
    key_df = pd.read_csv(value + '.csv')
    for index, row in key_df.iterrows():
        string = "INSERT INTO [AMZN_REV].[dbo].[{}] ([reviewerID],[asin],[helpful],[overall],[reviewTime]) values (?,?,?,?,?)".format(value)
        cursor.execute(string, row['reviewerID'], row['asin'], row['helpful'], row['overall'], row['reviewTime'])
        cursor.commit()

So here you have it.
Querying, creating tables & indexes and inserting values.

Basi
